## Packages & Jeu de données

In [5]:
import pandas as pd

# Chargement du jeu de données 
full_data = pd.read_csv('Data/TRAIN_FULL/BAAC_2012_to_2022_V3_to_V4.csv', low_memory=False)

full_data.head()

,place,catu,grav,sexe,trajet,an_nais,senc,catv,occutc,obs,...,nbv,prof,plan,surf,situ,secu1,secu2,secu3,motor,id_usager
0,1,1,0,1,5,1994,0.0,7,0.0,0.0,...,2,1,1,1,1,8,8,8,NaN,NaN
1,2,2,0,1,9,1992,0.0,7,0.0,0.0,...,2,1,1,1,1,1,8,8,NaN,NaN
2,1,1,0,1,5,1976,0.0,33,0.0,0.0,...,2,1,1,1,1,2,8,8,NaN,NaN
3,2,2,1,0,9,1972,0.0,33,0.0,0.0,...,2,1,1,1,1,2,8,8,NaN,NaN
4,1,1,0,0,5,1962,0.0,7,0.0,0.0,...,2,1,3,1,1,1,8,8,NaN,NaN


In [6]:
full_data.shape

(1012171, 35)

In [7]:
def convert_float_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    
def convert_float_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    data_frame[col_name] = data_frame[col_name].astype('category')
    
def convert_str_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype('category')
    
def convert_str_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)

### VEHICULES

Les fichiers véhicules contiennent globalement des informations relatives aux véhicules accidentés comme ...

#### senc

Le sens de circulation corrélé à certains endroits peut présenter un intérêt. Dans l'éventualité où cette variable serait pertinente, on décide de la retenir potentiellement pour la construction du modèle prédictif. Ainsi, on disposera des valeurs suivantes : [**1**, **2**, **3**]. La dernière valeur correspondra à la fois aux valeurs non renseignées, aux valeurs inconnues ou encore en cas d'absence de repère.

In [8]:
# Suppression de senc car trop vide
full_data.drop(labels=['senc'], axis=1, inplace=True)

#### catv

La catégorie du véhicule peut être un vecteur important de la gravité des accidents. À bicyclette, il est certainement plus rare de faire des accidents graves que lorsqu'un poids lourd de plusieurs tonnes vient percuter un piéton ou un autre automobiliste.

Dans un premier temps, on laisse la catégorisation initiale en pensant de manière candide que la segmentation des catégories de véhicules est qualitative. On se réserve le droit à l'avenir de traiter de nouveau cette catégorie pour construire un modèle plus précis ou moins sensible à l'overfitting en généralisant par exemple les poids lourds, les bus, les autocars...

In [9]:
# Harmonise les données
full_data['catv'] = full_data['catv'].replace({-1: 0})

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'catv')

1012171
1012171


#### obs, obsm

Ces deux variables sont liées aux obstacles fixes/mobiles heurtés. Ces renseignements sont primordiaux, en effet la connaissance d'un faussé peut être bien plus meurtrier au sein d'un accident (ou même un arbre), que la présence d'une glissière présente pour la sécurité.

In [10]:
# Harmonise les données
full_data['obs'] = full_data['obs'].replace({-1: pd.NA})
full_data['obsm'] = full_data['obsm'].replace({-1: pd.NA})
full_data = full_data.dropna(subset=['obs'])
full_data = full_data.dropna(subset=['obsm'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'obs')
convert_float_to_cat(full_data, 'obsm')

1012171
1011400


#### choc

Le point de choc peut certainement apporter des informations cruciales sur la gravité de l'accident. Certaines parties du véhicule peuvent amener à des conséquences plus ou moins importantes. En effet, de multiples chocs, fruits de tonneaux par exemple laisseront à coup sûr de gros dégâts au niveau du véhicule et potentiellment sur les accidentés.

In [12]:
# Harmonise les données
full_data['choc'] = full_data['choc'].replace({-1: pd.NA})
full_data = full_data.dropna(subset=['choc'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'choc')

#### manv

La manœuvre précédent l'accident est trés intéressante. On peut clairement relier le fait d'aller à contresens ou bien un franchissement de terre-plein central à un accident grave, tandis qu'un accident dans le même sens dans la même file peut réduire les dégâts (percuté par un automobiliste à un "cédez le passage" par exemple).

In [15]:
# Harmonise les données
full_data['manv'] = full_data['manv'].replace([-1, 0], pd.NA)
full_data = full_data.dropna(subset=['manv'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'manv')

C:\Users\lucas\AppData\Local\Temp\ipykernel_16704\3957277207.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data['manv'] = full_data['manv'].replace([-1, 0], pd.NA)


#### motor

Indique le type de motorisation du véhicule (électrique, hybride, etc.), cependant, cette catégorie comporte 65% de valeurs manquantes, car trés récente. Néanmoins, ce n'est pas un souci, parce que la catégorie du véhicule peut apporter de nombreuses informations sous-jacentes aux types de motorisation du véhicule.

In [16]:
# Suppression de la colonne motor 
full_data.drop(labels=['motor'], axis=1, inplace=True)

#### occutc

Correspond au nombre d'occupants dans le véhicule. Il n'est pas irrecevable qu'un lien existe entre cette valeur et la gravité de l'accident. Que ce soit pour des raisons de capacité de gestion de la part des dispositifs de sécurité, ou bien des comportements des conducteurs/passagers sous grande fréquentation. Malgré un grand nombre de valeurs manquantes, on juge la donnée assez pertinente pour se passer de 34% du jeu de données. On se le permet, car nous disposons de nombreuses observations (+1 000 000) et ce critère étant propre à toutes les années, il n'y aura théoriquement pas d'impact sur la pluralité des données.  

In [17]:
# Harmonise les données
full_data = full_data.dropna(subset=['occutc'])

# Converti en variable numérique
convert_float_to_int(full_data, 'occutc')

948764
581730


In [18]:
# Oublie de la suppression de id_usager
full_data.drop(labels=['id_usager'], axis=1, inplace=True)

In [19]:
full_data.to_csv('Data/TRAIN_FULL/BAAC_2012_to_2022_V4_to_V4.csv', index=False)

Désormais, toutes les variables propres aux usagers ont été traitées, on sauvegarde une version intermédiaire de ce fichier.